---
title: "Exploring income and physical activity disparities in the US"
subtitle: "INFO 511 - Fall 2024 - Final Project"
author: 
  - name: "The Outliers"
    affiliations:
      - name: "School of Information, University of Arizona"
description: "Using a dataset from the CDC of nutrition, physical activity, and obesity records, we will explore the possibility of a relationship between income and physical activity and examine its direction and strength."
format:
   html:
    code-tools: true
    code-overflow: wrap
    embed-resources: true
editor: visual
execute:
  warning: false
  echo: false
jupyter: python3
---

## Abstract

In this Final Project for INFO 511: Fundamentals of Data Science, we set out to investigate the potential relationship between physical activity and income level. Through analysis of historical health survey data, we attempt to better understand whether there is a visible trend or relationship between socioeconomic status and leisure-time physical activity in US adults.

## Introduction

Understanding the relationship between socioeconomic status and health behaviors is necessary for addressing disparities in public health outcomes. Our project seeks to understand whether higher-income populations consistently have more time for physical activity than lower income populations using a dataset from the Centers for Disease Control and Prevention (CDC). The dataset is specifically from the Behavioral Risk Factor Surveillance System project and was obtained from phone surveys conducted between 2011 and 2023. The whole dataset offers insights into physical activity, nutrition, and obesity trends among U.S. residents aged 18 and older. For the purpose of this project, we are focusing on the survey questions related to physical activity. The data is stratified by factors such as age, education, gender, income, and race/ethnicity.

## Research Question

**Do higher-income populations have more time for physical activity than lower income populations?**

We hypothesize that this is true, higher income populations have more time for physical activity. Therefore, populations will engage in more physical activity as their income level increases (positive relationship).

## Data

Dataset: [Nutrition, Physical Activity, and Obesity - Behavioral Risk Factor Surveillance System](https://chronicdata.cdc.gov/Nutrition-Physical-Activity-and-Obesity/Nutrition-Physical-Activity-and-Obesity-Behavioral/hn4x-zwk7/about_data)

This dataset is hosted by the United States Center for Disease Control and was obtained from the Behavioral Risk Factor Surveillance System, a CDC project consisting of health-related phone surveys. The original dataset consists of 104,000 rows and 33 columns. Descriptions of all columns are available on the link above. Each row represents a combination of a year, state, survey question, and percent of individuals who are positively identified for that question, along with stratification. Data_Value contains the corresponding value collected for each survey question. The categories for stratification are Age Range, Education, Gender, Income, Race/Ethnicity, and Total. This dataset includes observations for the years 2011-2023. Percentages and data are not included for groups with insufficient sample sizes.

The main columns of interest for our research question are:

-   YearStart and YearEnd: The year the data was collected. These are the same for every row.
-   LocationAbbr and LocationDesc: Contains the abbreviation for the data where the data was collected.
-   Topic: Contains the topic the variable being measured falls into. For our research question, we are interested in the topic "Physical Activity - Behavior"
-   Question: What is being measured. Within "Physical Activity - Behavior" there are 5 questions:
    -   Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic physical activity and engage in muscle-strengthening activities on 2 or more days a week
    -   Percent of adults who engage in no leisure-time physical activity
    -   Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)
    -   Percent of adults who engage in muscle-strengthening activities on 2 or more days a week
    -   Percent of adults who achieve at least 300 minutes a week of moderate-intensity aerobic physical activity or 150 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)
-   Data_Value: The value being measured by the survey, in this case for these specific questions it will be a percentage.
-   StratificationCategory1: What variable the data is being stratified by. Depending on the value in this column, it will contain a value in the columns "Race", "Age (years)", "Income", etc. For our research question we are interested in the levels in Income, such as 'Less than \$15,000', '\$35,000 - \$49,999', etc.
-   Income: Contains the income level.


In [ ]:
import pandas as pd
data = pd.read_csv('data/Nutrition.csv')
data.head()

In [ ]:
data.info()

### Data Cleaning and Wrangling, EDA

The columns YearStart and YearEnd always contain the same values so one of these columns can be dropped. We are only interested in the rows containing questions related to physical activity. Specifically, we are interested in the rows corresponding to the value in column `Question` that describes: "Percent of adults who engage in no leisure-time physical activity". There were five measurements collected related to physical activity from the phone surveys from which the dataset was derived, though not all of these questions were asked every year. The following statements are how the measurements were described in the `Question` column, but were not the way the questions were presented to participants over the phone. The exact wording of the questions is available on the [Behavioral Risk Factor Surveillance System website](https://www.cdc.gov/brfss/questionnaires/index.htm).

1.  Percent of adults who engage in no leisure-time physical activity

2.  Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)

3.  Percent of adults who engage in muscle-strengthening activities on 2 or more days a week

4.  Percent of adults who achieve at least 300 minutes a week of moderate-intensity aerobic physical activity or 150 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)

5.  Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic physical activity and engage in muscle-strengthening activities on 2 or more days a week

There are multiple columns containing the categories for stratification, such as education levels in the column `Education`. To answer our research question, we are most interested in the `Income` column. The data can also be separated by US state using the `LocationAbbr` (state abbreviation, i.e. "AZ") or `LocationDesc` (full state text, i.e. "Arizona") columns. There are also national measurements, using the abbreviation "US" or full text "National".

Some of the columns contained missing values. Missing values were dropped for the column `Data_Value` because the percentage values in this column were the focus of our analysis, and we were not interested in the years where this question was not measured. One abnormality of note in this dataset was the column `Data_Value_Unit`. This column was described as containing the unit of measurement for `Data_Value`, but this was not the case as the column contained years followed by a period. This may be indicative of a data entry error.


In [ ]:
data.loc[data['YearStart'] != data['YearEnd']]

In [ ]:
activity = data.drop(columns='YearEnd')

In [ ]:
activity = activity.loc[activity['Topic'] == 'Physical Activity - Behavior']
activity.drop(columns=["Total", "Education", "Age(years)", "Gender", "Race/Ethnicity", "GeoLocation"], inplace=True)
activity = activity[activity['StratificationCategory1'] == "Income"]
activity.info()

#### Encoding income as a numeric value

The income level of the participants is encoded in the dataset as a string value. The income levels were placed into the following bins:

-   Less than \$15,000

-   \$15,000 - \$24,999

-   \$25,000 - \$34,999

-   \$35,000 - \$49,999

-   \$50,000 - \$74,999

-   \$75,000 or greater

This can be used as a nominal ordinal variable, but to do linear regression this must be converted to a numeric variable. For this project, we encoded the income as the first number in the range, such as 15,000 for \$15,000 - \$24,999.


In [ ]:
# maps income ranges to numeric values
income_dict = {'Less than $15,000':0,
  '$15,000 - $24,999':15,
  '$25,000 - $34,999':25, 
  "$35,000 - $49,999":35, 
  '$50,000 - $74,999':50, 
  '$75,000 or greater':75,
        }

# removes data not reported
activity_clean = activity.loc[activity['Income'] != 'Data not reported']

# creates numeric column of income based on mappings
activity_clean['numeric_income'] = activity_clean['Income'].replace(income_dict)

## Visualization of Dataset

### Overview of National Levels of Leisure-Time Physical Activity


In [ ]:
# onyl include question of interest
noleisuretime = activity_clean[activity_clean['Question']=='Percent of adults who engage in no leisure-time physical activity']

# Stratified by income
noleisuretime_income = noleisuretime[noleisuretime['StratificationCategory1']=='Income']

# Only include national data, disregarding results from individual states
noleisuretime_income_national = noleisuretime_income[noleisuretime_income['LocationDesc']=="National"]


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#manual order for income levels
order = ['Less than $15,000',
  '$15,000 - $24,999',
  '$25,000 - $34,999', 
  "$35,000 - $49,999", 
  '$50,000 - $74,999', 
  '$75,000 or greater']

# order data 
noleisuretime_income_national['Income'] = pd.Categorical(noleisuretime_income_national['Income'],categories=order, ordered=True)

# plot
sns.set_style('whitegrid')
plt.figure(figsize=(8,6))

sns.violinplot(data=noleisuretime_income_national, x="Income", y="Data_Value", palette='viridis')
plt.xticks(rotation=-20)
plt.ylabel('Percent')
plt.title('From 2011-Percent of US Adults Who Do Not Engage in Leisure-Time Physical Activity')
plt.text(2.1, -1, 'Source: CDC, Behavioral Risk Factor Surveillance System (2011 - 2023)', fontsize = 8)

plt.show()

This violin plot shows the distribution and density of the data for each income group, with the y-axis representing the percentage of adults not engaging in physical activity and the x-axis denoting income categories. The lowest income category, "Less than \$15,000", has the highest percentage of adults who do not engage in any leisure-time physical activity, and the opposite is true for the highest income category, "\$75,000 or greater", which has the most adults who do engage in leisure-time physical activity.

This plot provides a general overview of the trends in the physical activity of American adults as related to income. Further analysis of this dataset shows the same association between higher income and higher percentages of adults who engage in physical activity.


In [ ]:
question_mapping = {
    'Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic physical activity and engage in muscle-strengthening activities on 2 or more days a week': 'Moderate Exercise & Muscle Strengthening',
    'Percent of adults who engage in muscle-strengthening activities on 2 or more days a week': 'Muscle Strengthening',
    'Percent of adults who engage in no leisure-time physical activity': 'No Leisure-time Physical Activity',
    'Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)': 'Moderate Exercise',
    'Percent of adults who achieve at least 300 minutes a week of moderate-intensity aerobic physical activity or 150 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)': 'Intense exercise'
}

activity_clean['Short_Question'] = activity_clean['Question'].map(question_mapping)
activity_clean['Question'].map(question_mapping)

national = activity_clean[activity_clean['LocationDesc'] == 'National']

g = sns.FacetGrid(national, col="Short_Question", hue='Income', hue_order=order, col_wrap=3, aspect=1.5, palette='viridis')
g.map(sns.regplot, 'YearStart', 'Data_Value', )

# Rotate the x-axis labels for better readability
for ax in g.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)

# Enhancing the plot with titles and labels
g.set_titles('{col_name}')  # Set titles to be each question
g.set_axis_labels('Year', 'Percent of Pop')  # Set common axis labels

# Adjust the top space to accommodate title
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Percent of Population for Each Exercise Class Over Years')

# Adding the legend
g.add_legend(title='Income Bracket')
# Show the plot
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression

sns.set_style(style='whitegrid')
sns.scatterplot(data=noleisuretime, x='numeric_income', y='Data_Value', alpha=0.1)

model = LinearRegression()
x = no_leisure_time[['numeric_income']]
y = no_leisure_time['Data_Value']
model.fit(x, y)

slope = model.coef_[0]
intercept = model.intercept_
r_squared = model.score(x, y)

x_range = np.linspace(x.min(), x.max(), 100)  # Range of x values
y_pred = model.predict(x_range)  # Predicted y values
plt.plot(x_range, y_pred, color='red', label=f"y = {slope:.2f}x + {intercept:.2f}, $R^2$ = {r_squared:.2f}")

plt.title("Percent of sample population who engage in no leisure time activity", fontsize=14)
plt.xlabel("Minimum Income in Range (i.e. 15,000 for \$15,000 - \$24,999)", fontsize=12)
plt.ylabel("Percent of Sample Population", fontsize=12)
plt.legend()
plt.show()